# 1. Parameters

In [1]:
cases_dir = 'cases/unset'
metadata_file = 'input/metadata-subsample-pangolin.tsv'

In [2]:
# Parameters
cases_dir = "cases/case-100"
iterations = 3
number_samples = 100


In [3]:
from pathlib import Path
import imp
fp, pathname, description = imp.find_module('gdi_benchmark', ['../../lib'])
gdi_benchmark = imp.load_module('gdi_benchmark', fp, pathname, description)

cases_dir_path = Path(cases_dir)

case_name = str(cases_dir_path.name)
index_path = cases_dir_path / 'index'

output_api_path = cases_dir_path / 'query-api.tsv'
output_cli_path = cases_dir_path / 'query-cli.tsv'

# 2. Benchmark command-line

In [4]:
import pandas as pd
import genomics_data_index.api as gdi

def benchmark_cli_index(name: str, index_path: Path) -> pd.DataFrame:
    benchmark_commands = {
        'query hasa':               f'gdi --project-dir {index_path} --ncores 1 query "hasa:hgvs_gn:NC_045512.2:S:p.D614G"',
        'query isa':                f'gdi --project-dir {index_path} --ncores 1 query "isa:Switzerland/100112/2020"',
        'query isin':               f'gdi --project-dir {index_path} --ncores 1 query --reference-name NC_045512 "isin_5_substitutions:Switzerland/100112/2020"',
        'query --summary':          f'gdi --project-dir {index_path} --ncores 1 query --summarize',
        'query --features-summary': f'gdi --project-dir {index_path} --ncores 1 query --features-summary',
        'list samples':             f'gdi --project-dir {index_path} --ncores 1 list samples',
    }
    
    db = gdi.GenomicsDataIndex.connect(index_path)
    number_samples = db.count_samples()
    number_features_no_unknown = db.count_mutations(reference_genome='NC_045512', include_unknown=False)
    number_features_all = db.count_mutations(reference_genome='NC_045512', include_unknown=True)
    iterations = 10
    
    benchmarker = gdi_benchmark.QueryBenchmarkHandler()
    return benchmarker.benchmark_cli(name=name, kind_commands=benchmark_commands, number_samples=number_samples,
                                    number_features_no_unknown=number_features_no_unknown, number_features_all=number_features_all,
                                     iterations=iterations)

In [5]:
cli_df = benchmark_cli_index(name=case_name, index_path=index_path)
cli_df.head(3)

,Name,Kind,Iteration,Number samples,Number features (no unknown),Number features (all),Runtime,Memory (max),Mmemory (max/process)
0,case-100,query hasa,1,100,370,9536,4.31,221790208.0,221790208.0
0,case-100,query hasa,2,100,370,9536,3.96,238583808.0,222044160.0
0,case-100,query hasa,3,100,370,9536,5.03,237809664.0,221376512.0


In [6]:
cli_df.to_csv(output_cli_path, sep='\t', index=False)

# 3. Test query API

## 3.1. Load (example) metadata

The simulated data is based off of real sample names and a real tree. So I can load up real metadata and attach it to a query (though the mutations and reference genome are all simulated).

In [7]:
import pandas as pd

metadata_df = pd.read_csv(metadata_file, sep='\t')
metadata_df.head(2)

,strain,virus,gisaid_epi_isl,genbank_accession,sra_accession,date,region,country,division,location,...,ambiguity_score,scorpio_call,scorpio_support,scorpio_conflict,version,pangolin_version,pangoLEARN_version,pango_version,status,note
0,England/QEUH-148725F/2021,ncov,EPI_ISL_1487327,OU224813,ERR5679447,2021-03-28,Europe,United Kingdom,England,NaN,...,1.0,Alpha (B.1.1.7-like),0.9565,0.0435,PLEARN-v1.2.66,3.1.11,2021-08-24,v1.2.66,passed_qc,scorpio call: Alt alleles 22; Ref alleles 1; A...
1,England/MILK-1535AC0/2021,ncov,EPI_ISL_1985652,OU266415,ERR5905912,2021-04-24,Europe,United Kingdom,England,NaN,...,1.0,B.1.1.318-like,0.8421,0.0000,PLEARN-v1.2.66,3.1.11,2021-08-24,v1.2.66,passed_qc,scorpio call: Alt alleles 16; Ref alleles 0; A...


## 3.2. Define benchmark cases

In [8]:
from typing import List
import genomics_data_index.api as gdi

def benchmark_api_index(name: str, index_path: Path) -> pd.DataFrame:
    db = gdi.GenomicsDataIndex.connect(index_path)
    q_no_join = db.samples_query(reference_name='NC_045512', universe='mutations')
    q_join = db.samples_query(reference_name='NC_045512', universe='mutations').join(metadata_df, sample_names_column='strain')

    q = q_join.hasa('hgvs_gn:NC_045512.2:S:p.D614G')
    r = q_join.hasa('hgvs_gn:NC_045512.2:N:p.R203K')
    
    number_samples = db.count_samples()
    number_features_no_unknown = db.count_mutations(reference_genome='NC_045512', include_unknown=False)
    number_features_all = db.count_mutations(reference_genome='NC_045512', include_unknown=True)
    repeat = 10

    benchmark_cases = {
        'db.samples_query':      lambda: db.samples_query(reference_name='NC_045512', universe='mutations'),
        'q.join':                lambda: q_no_join.join(metadata_df, sample_names_column='strain'),
        'q.features_summary':    lambda: q_join.features_summary(),
        'q.features_comparison': lambda: q_join.features_comparison(sample_categories='lineage', categories_kind='dataframe', kind='mutations', unit='proportion'),
        'q.hasa':                lambda: q_join.hasa("hgvs_gn:NC_045512.2:N:p.R203K"),
        'q.isa':                 lambda: q_join.isa("Switzerland/100112/2020"),
        'q.isin (distance)':     lambda: q_join.isin("Switzerland/100112/2020", kind='distance', distance=5, units='substitutions'),
        'q.isin (mrca)':         lambda: q_join.isin(["Switzerland/100112/2020", "LR880642"], kind='mrca'),
        'q AND r':               lambda: q & r,
        'q.toframe':             lambda: q_join.toframe(),
        'q.summary':             lambda: q_join.summary(),
    }

    benchmarker = gdi_benchmark.QueryBenchmarkHandler()
    return benchmarker.benchmark_api(name=name, kind_functions=benchmark_cases,
                                       number_samples=number_samples,
                                       number_features_no_unknown=number_features_no_unknown,
                                       number_features_all=number_features_all,
                                       repeat=repeat)

## 3.3. Benchmark reads index

In [9]:
api_df = benchmark_api_index(name=case_name, index_path=index_path)
api_df.head(5)

Attempting to set global database_path_translator=<genomics_data_index.storage.model.db.DatabasePathTranslator.DatabasePathTranslator object at 0x7f82291deac0> but it is already set


,Name,Kind,Number samples,Number features (no unknown),Number features (all),Number executions,Iteration,Time
0,case-100,db.samples_query,100,370,9536,100,1,0.003631
1,case-100,db.samples_query,100,370,9536,100,2,0.003623
2,case-100,db.samples_query,100,370,9536,100,3,0.004573
3,case-100,db.samples_query,100,370,9536,100,4,0.003609
4,case-100,db.samples_query,100,370,9536,100,5,0.003599


In [10]:
api_df.to_csv(output_api_path, sep='\t', index=False)